<a href="https://colab.research.google.com/github/Ivan-KKH/APAI4011_GP/blob/master/predicting_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import numpy as np
import pandas as pd
import io
import re
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Bidirectional
import warnings
import random
warnings.filterwarnings("ignore")

In [ ]:
uploaded = files.upload()

Saving amazon.csv to amazon.csv


In [ ]:
#read csv file
df = pd.read_csv(io.BytesIO(uploaded['amazon.csv']))
#dfReview for review title and review content
dfText = df[['product_name','about_product','category']]

In [ ]:
for column in range(2):
  dfText.iloc[:,column] = dfText.iloc[:,column].apply(lambda x: x.lower())
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub('[^a-zA-z0-9\s]'," ",x)))
dfText['category'] = dfText['category'].apply(lambda x: x.split('|',1)[0])

#combine product_name and about_product
dfText['combined'] = dfText['product_name'] + ' ' + dfText['about_product']

In [ ]:
#tokenize, build vocab, and add paddings on product_name
tokenizer = Tokenizer(num_words=2000,split=' ')
tokenizer.fit_on_texts(dfText['combined'].values)
text = tokenizer.texts_to_sequences(dfText['combined'].values)
text = pad_sequences(text)

#category
category_list = []
for categ in dfText['category']:
  if categ not in category_list:
    category_list.append(categ)
print(f'category list: {category_list}')
category = pd.get_dummies(dfText['category'], columns=category_list).values
category_list = ['Car & Motorbike', 'Computers & Accessories', 'Electronics', 'Health & Personal Care', 'Home & Kitchen', 'Home Improvement', 'Musical Instruments', 'Office Products', 'Toys & Games']

print(f'shape of text: {text.shape}')
print(f'shape of category: {category.shape}')


category list: ['Computers&Accessories', 'Electronics', 'MusicalInstruments', 'OfficeProducts', 'Home&Kitchen', 'HomeImprovement', 'Toys&Games', 'Car&Motorbike', 'Health&PersonalCare']
shape of text: (1465, 424)
shape of category: (1465, 9)


In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Flatten, Reshape, SpatialDropout1D
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
#split data
X_train, X_test, Y_train, Y_test = train_test_split(text, category, test_size=0.2, random_state=42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

#apply Bidirectional LSTM
model = Sequential()
model.add(Embedding(2000, 128, input_length = 424))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(196, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(9,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

#Train the model for 5 epoches with embedding matrix you obtained earlier.
#print training loss and training accuracy for each epoch
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test,Y_test))  


(1172, 424) (1172, 9)
(293, 424) (293, 9)


Epoch 1/10
37/37 [==============================] - 158s 4s/step - loss: 1.3598 - accuracy: 0.3507 - val_loss: 1.0952 - val_accuracy: 0.4949
Epoch 2/10
37/37 [==============================] - 146s 4s/step - loss: 0.8389 - accuracy: 0.7304 - val_loss: 0.9052 - val_accuracy: 0.6553
Epoch 3/10
37/37 [==============================] - 147s 4s/step - loss: 0.5892 - accuracy: 0.8302 - val_loss: 0.5167 - val_accuracy: 0.8771
Epoch 4/10
37/37 [==============================] - 156s 4s/step - loss: 0.3154 - accuracy: 0.9104 - val_loss: 0.4675 - val_accuracy: 0.8703
Epoch 5/10
37/37 [==============================] - 146s 4s/step - loss: 0.3814 - accuracy: 0.9138 - val_loss: 0.8461 - val_accuracy: 0.7167
Epoch 6/10
37/37 [==============================] - 163s 4s/step - loss: 0.3567 - accuracy: 0.8942 - val_loss: 0.3941 - val_accuracy: 0.9078
Epoch 7/10
37/37 [==============================] - 147s 4s/step - loss: 0.1417 - accuracy: 0.9590 - val_loss: 0.2989 - val_accuracy: 0.9283
Epoch 8/10
37

In [ ]:
#auto category tagger function
#choose a random product
rand_idx = random.randint(0,len(df))
product_name = dfText['product_name'][rand_idx]
about_product = dfText['about_product'][rand_idx]

#reshape the tokenised input
input= np.reshape(text[rand_idx],(1, text[rand_idx].shape[0]))

#predict the category
pred = model.predict(input)
max_idx = np.argmax(pred)
predicted_category = category_list[max_idx]

#print out the results
actual_category = np.argmax(category[rand_idx])
print(f"Product name: {product_name}")
print(f"Product description: {about_product}")
print(f'Predicted Category: {predicted_category}')
print(f'Actual Category: {category_list[actual_category]}')

1/1 [==============================] - 0s 242ms/step
Product name: sandisk ultra dual drive go usb type c pendrive for mobile  black  128 gb  5y   sdddc3 128g i35 
Product description: the 2 in 1 flash drive with a reversible usb type c and a traditional type a connector  seamlessly move content between your usb type c smartphone  tablets and macs and usb type a computers  free up space on your usb type c smartphone so you can take more photos  automatically back up photos using the sandisk memory zone app high performance usb 3 1 gen 1 drive with 150mb s read speeds lets you quickly move your files to your computer 1 dual purpose swivel design protects connectors and features a keyring hole to take your drive on the go 
Predicted Category: Computers & Accessories
Actual Category: Computers & Accessories
